In [ ]:
import numpy as np
import pandas as pd
import multiprocessing
from time import time
from matplotlib import pyplot as plt
%load_ext Cython

In [ ]:
name = "data/05250115"

print('Reading',name+'_MCP.csv','...')
%time MCP_original_dataframe = pd.read_csv(name+'_MCP.csv')
print('done.')
print()
print('Reading',name+'_CsI.csv','...')
%time CsI_original_dataframe = pd.read_csv(name+'_CsI.csv')
print('done.')
print()
print('Reading',name+'_MWPC.csv','...')
%time MWPC_original_dataframe = pd.read_csv(name+'_MWPC.csv')
print('done.')

In [ ]:
# Select event that hit 3~5 MWPCs.
# Todo: also select hit CsI.
MWPC_hit_count = MWPC_original_dataframe.SN.value_counts()
data_SN = MWPC_hit_count[(3 <= MWPC_hit_count) & (MWPC_hit_count <= 5)].index.sort_values()
effective_event_num = data_SN.shape[0]
print('Raw data consists of', MCP_original_dataframe.shape[0], 'events.')
print('After filtering,', effective_event_num, 'events will be retained.')

# Raw data in ndarray form.
# Note:
# The first subscript of this ndarray points to the event(0~n),
# The second subscript points to SN(0) and the data of the three detectors MCP, CsI, and MWPC in this event(1~3),
# The third subscript points to one of the records of the detector in this event 
# (Here, there should be only one record for MCP and CsI, while MWPC can have several records).
# The fourth subscript points to the specific value of this record, which corresponds to each column in the original data. 
raw_data = np.ndarray((effective_event_num, 4), dtype=object)

print('Filling SN ...', end=' ')
for i in range(effective_event_num):
    raw_data[i][0] = data_SN[i]
print('done.')

def FilterAndConvertMCP(MCP_original_dataframe, data_SN, pipe_MCP):
    print('worker_MCP > Filtering and converting MCP data ...')
    time_start = time()
    MCP_data = np.ndarray(effective_event_num, dtype=object)
    MCP_data.fill(np.ndarray(0, dtype=object))
    i = 0
    for event in MCP_original_dataframe[MCP_original_dataframe.SN.isin(data_SN)].groupby('SN'):
        while data_SN[i] != event[0]: i += 1
        MCP_data[i] = event[1].values.T[1:4].T
    print('worker_MCP > done. Elapsed time:', time()-time_start, 's')
    print('worker_MCP > MCP_data has sent.')
    pipe_MCP.send(MCP_data)
    pipe_MCP.close()

def FilterAndConvertCsI(CsI_original_dataframe, data_SN, pipe_CsI):
    print('worker_CsI > Filtering and converting CsI data ...')
    time_start = time()
    CsI_data = np.ndarray(effective_event_num, dtype=object)
    CsI_data.fill(np.ndarray(0, dtype=object))
    i = 0
    for event in CsI_original_dataframe[CsI_original_dataframe.SN.isin(data_SN)].groupby('SN'):
        while data_SN[i] != event[0]: i += 1
        CsI_data[i] = event[1].values.T[1:3].T
    print('worker_CsI > done. Elapsed time:', time()-time_start, 's')
    print('worker_CsI > CsI_data has sent.')
    pipe_CsI.send(CsI_data)
    pipe_CsI.close()

def FilterAndConvertMWPC(MWPC_original_dataframe, data_SN, pipe_MWPC):
    print('worker_MWPC > Filtering and converting MWPC data ...')
    time_start = time()
    MWPC_data = np.ndarray(effective_event_num, dtype=object)
    MWPC_data.fill(np.ndarray(0, dtype=object))
    i = 0
    for event in MWPC_original_dataframe[MWPC_original_dataframe.SN.isin(data_SN)].groupby('SN'):
        while data_SN[i] != event[0]: i += 1
        MWPC_data[i] = event[1].values.T[1:5].T
    print('worker_MWPC > done. Elapsed time:', time()-time_start, 's')
    print('worker_MWPC > MWPC_data has sent.')
    pipe_MWPC.send(MWPC_data)
    pipe_MWPC.close()

pipe_MCP_parent, pipe_MCP = multiprocessing.Pipe()
worker_MCP = multiprocessing.Process(target=FilterAndConvertMCP, args=(MCP_original_dataframe, data_SN, pipe_MCP))
worker_MCP.start()

pipe_CsI_parent, pipe_CsI = multiprocessing.Pipe()
worker_CsI = multiprocessing.Process(target=FilterAndConvertCsI, args=(CsI_original_dataframe, data_SN, pipe_CsI))
worker_CsI.start()

pipe_MWPC_parent, pipe_MWPC = multiprocessing.Pipe()
worker_MWPC = multiprocessing.Process(target=FilterAndConvertMWPC, args=(MWPC_original_dataframe, data_SN, pipe_MWPC))
worker_MWPC.start()

raw_data.T[1] = pipe_MCP_parent.recv()
print('MCP_data recieved.')
raw_data.T[2] = pipe_CsI_parent.recv()
print('CsI_data recieved.')
raw_data.T[3] = pipe_MWPC_parent.recv()
print('MWPC_data recieved.')

worker_MCP.join()
worker_CsI.join()
worker_MWPC.join()

In [ ]:
# %%cython
# import numpy as np
# cimport numpy as np

def FindHelix(MWPC_event, gamma_GD=True):
    c = 299.79        # c, mm/ns
    m0_eB = 0.056778  # m0/eB, ns, >0 if e>0, <0 if e<0
    gamma = 103.72    # Lorentz factor for 53MeV
    if gamma_GD:
        # nondimensionalizing
        r_dimless = MWPC_event.T[1:4].T / (m0_eB * c)
        # fit helix with fixed gamma.
        max_epochs = 20
        fit_result = FitHelixGammaGD(r_dimless, gamma, max_epochs)
    else:
        # nondimensionalizing
        t_dimless = MWPC_event.T[0] / (m0_eB)
        r_dimless = MWPC_event.T[1:4].T / (m0_eB * c)
        # fit helix and gamma.
        fit_result = FitHelixGammaFixed(t_dimless, r_dimless, gamma)
    return fit_result

# r should be nondimensionalized.
# assumming r is sorted by distance to origin point.
def FitHelixGammaGD(r, gamma, max_epochs=20, alpha=0.1, ep=1e-6):
    # initialize
    loss = 0
    paras = np.array([0, 0, 0, gamma, 0])
    # GD
    last_loss = loss
    r2, loss, paras = Cost(paras, gamma, r)
    gamma -= alpha * GradCost(paras, gamma, r)
    epochs_count = 1
    while (epochs_count < max_epochs) & (np.abs(last_loss/loss - 1) > ep):
        last_loss = loss
        r2, loss, paras = Cost(paras, gamma, r)
        gamma -= alpha * GradCost(paras, gamma, r)
        epochs_count += 1
    return np.array([r2, loss, paras, gamma, epochs_count], dtype=object)

# all paras should be nondimensionalized.
# assumming r is sorted by distance to origin point.
def Cost(paras, gamma, r):
    t = np.zeros(r.shape[0])
    for i in range(1, t.shape[0]):
        arc = paras[3] * (np.arctan((r[i][0]-paras[0])/(r[i][1]-paras[1])) - np.arctan((r[i-1][0]-paras[0])/(r[i-1][1]-paras[1])))
        lz = r[i][2] - r[i-1][2]
        t[i] = t[i-1] + np.sqrt((arc*arc + lz*lz) / (1 - 1/(gamma*gamma)))
    return FitHelixGammaFixed(t, r, gamma)

# all paras should be nondimensionalized.
# assumming r is sorted by distance to origin point.
def GradCost(paras, gamma, r, h=1e-6):
    loss1 = Cost(paras, gamma-h, r)[1]
    loss2 = Cost(paras, gamma+h, r)[1]
    return (loss2 - loss1) / (2*h)

# all paras should be nondimensionalized.
def FitHelixGammaFixed(t, r, gamma):
    cos_t_gamma = np.cos(t/gamma)
    sin_t_gamma = np.sin(t/gamma)
    x = r.T[0]
    y = r.T[1]
    z = r.T[2]
    n = t.shape[0]
    # fit t->(x,y)
    sum_sin_t_gamma = sin_t_gamma.sum()
    sum_cos_t_gamma = cos_t_gamma.sum()
    A = np.array([
        [ n,               0,                sum_cos_t_gamma,  sum_sin_t_gamma ],
        [ 0,               n,                -sum_sin_t_gamma, sum_cos_t_gamma ],
        [ sum_cos_t_gamma, -sum_sin_t_gamma, n,                0               ],
        [ sum_sin_t_gamma, sum_cos_t_gamma,  0,                n               ]
    ])
    b = np.array([x.sum(), y.sum(), (x*cos_t_gamma-y*sin_t_gamma).sum(), (x*sin_t_gamma+y*cos_t_gamma).sum()])
    x0,y0,Rx,Ry = np.linalg.solve(A,b)
    # fit t->z
    t_mean = t.mean()
    z_mean = z.mean()
    vz = ((t*z).sum() - n*t_mean*z_mean) / ((t*t).sum() - n*t_mean*t_mean)
    z0 = z_mean - vz*t_mean
    
    # scoring
    r_hat = np.ndarray((t.shape[0],3))
    for i in range(t.shape[0]):
        r_hat[i] = Helix(t[i], cos_t_gamma[i], sin_t_gamma[i], x0, y0, z0, Rx, Ry, vz)
    r = np.array([x, y, z]).T
    r_mean = np.array([x.mean(), y.mean(), z.mean()])
    d_hat = r_hat - r
    d_mean = r_mean - r
    sse = 0
    for d_hat_i in d_hat:
        sse += np.dot(d_hat_i, d_hat_i)
    sst = 0
    for d_mean_i in d_mean:
        sst += np.dot(d_mean_i, d_mean_i)
    r2 = 1 - sse / sst
    
    # # plotting
    # # plot x y
    # plt.plot(r_hat.T[0], r_hat.T[1], c='r', ls=':')
    # plt.scatter(x, y)
    # plt.xlabel(r'$x\left/\frac{m_0 c}{eB}\right.$', fontsize=14)
    # plt.ylabel(r'$y\left/\frac{m_0 c}{eB}\right.$', fontsize=14)
    # plt.show()
    # # plot full x y
    # t_plot = np.linspace(0, 6.28, 100)
    # r_plot = np.ndarray((100,3))
    # for i in range(r_plot.shape[0]):
    #     r_plot[i] = Helix(t_plot[i], np.cos(t_plot[i]), np.sin(t_plot[i]), x0, y0, z0, Rx, Ry, vz)
    # plt.plot(r_plot.T[0], r_plot.T[1], c='r', ls=':')
    # plt.scatter(x, y)
    # plt.scatter(x0, y0)
    # plt.xlabel(r'$x\left/\frac{m_0 c}{eB}\right.$', fontsize=14)
    # plt.ylabel(r'$y\left/\frac{m_0 c}{eB}\right.$', fontsize=14)
    # plt.show()
    # # plot t z
    # plt.plot(t, r_hat.T[2], c='r', ls=':')
    # plt.scatter(t, z)
    # plt.xlabel(r'$\frac{eB}{m_0}t$', fontsize=14)
    # plt.ylabel(r'$z\left/\frac{m_0 c}{eB}\right.$', fontsize=14)
    # plt.show()
    
    return np.array([r2, sse, np.array([x0, y0, z0, np.sqrt(Rx*Rx + Ry*Ry), vz])], dtype=object)
    
# all paras should be nondimensionalized.
def Helix(t, cos_t_gamma, sin_t_gamma, x0, y0, z0, Rx, Ry, vz):
    r = np.ndarray(3)
    r[0] = x0 + Rx*cos_t_gamma + Ry*sin_t_gamma
    r[1] = y0 + Ry*cos_t_gamma - Rx*sin_t_gamma
    r[2] = z0 + vz*t
    return r

In [ ]:
FindHelix(raw_data[7][3], True)

In [ ]:
# %%cython
# import numpy as np
# cimport numpy as np

def FindTOFAndRdca(raw_event):
    # SN Rdca TOF TOF_E
    RdcaAndTOF = np.ndarray(3, dtype=object)
    # SN.
    RdcaAndTOF[0] = raw_event[0]
    
    RdcaAndTOF[2] = FindHelix(raw_event[3])
    return RdcaAndTOF

In [ ]:
# pool = multiprocessing.Pool()
# %time fit_result = np.array(pool.map(FindTOFAndRdca, raw_data[0:4000]), dtype=object)
# pool.close()
# pool.join()